In [2]:
# !pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 43.6 MB/s eta 0:00:00


In [2]:
import os
from typing import IO
import gradio as gr
import requests
from tqdm import tqdm
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


# 생성형 이미지 만들기 UI 구현

In [3]:
WIDTH = 512
HEIGHT = 512

with gr.Blocks() as app:
    gr.Markdown("## 프롬프트 입력")
    with gr.Row():
        prompt = gr.Textbox(label="Prompt")
    with gr.Row():
        prompt = gr.Textbox(label="Nagative Prompt")

    gr.Markdown("## 스케치 to 이미지 생성")
    with gr.Row():
        with gr.Column():
            with gr.Tab("Canvas"):
                canvas = gr.Image(
                    label="Draw",
                    source='canvas',
                    image_mode='RGB',
                    tool='color-sketch',
                    interactive=True,
                    width=WIDTH,
                    height=HEIGHT,
                    shape=(WIDTH, HEIGHT),
                    brush_radius=20,
                    type='pil'
                )
            with gr.Row():
                canvas_run_btn = gr.Button(value="Generate")
                
            with gr.Tab("File"):
                canvas = gr.Image(
                    label="upload",
                    source='upload',
                    image_mode='RGB',
                    tool='color-sketch',
                    interactive=True,
                    width=WIDTH,
                    height=HEIGHT,
                    shape=(WIDTH, HEIGHT),
                    brush_radius=20,
                    type='pil'
                )
            with gr.Row():
                file_run_btn = gr.Button(value="Generate")

app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://0ed2c7ac0cc289b0d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [6]:
# pip install gradio==3.40.0

Note: you may need to restart the kernel to use updated packages.


In [8]:
app.close()

Closing server running on port: 7861


# 모델 다운로드 ui 구현

In [35]:
with gr.Blocks() as app:
    gr.Markdown("## 모델 다운로드")
    with gr.Row():
        model_url = gr.Textbox(label="모델 URL", placeholder="https://civitai.com/")
        download_model_btn = gr.Button(value="모델 다운로드")
    with gr.Row():
        model_file = gr.File(label="모델 파일")

    download_model_btn.click(
        download_model,
        [model_url],
        [model_file]
    )
    
app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7872
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://9cd058d889881a5ccc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/blocks.py", line 1427, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/anyio/_backends/_asyncio.py", line 

In [15]:
app.close()

Closing server running on port: 7864


# 모델 다운로드 기능 구현

In [34]:
import os
import glob

# 전역 변수로 모델 경로와 파일명을 저장
MODEL_PATH=None

# URL로부터 파일 다운로드 함수
def download_from_url(url, file_path, chunk_size=1024):
    try:
        resp = requests.get(url, stream=True)
        resp.raise_for_status()
        
    except Exception as e:
        print(f"[Error]{e}")
        raise e
        
    total = int(resp.headers.get('content-length',0)) #파일 크기 추출
    with open(file_path, 'wb') as file, tqdm(desc=file_path, total = total, unit='iB', unit_scale=True, unit_divisor = 1024) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)
        
        
#모델을 다운로드하고 경로를 기억하는 함수
def download_model(url: str) -> str:
    global MODEL_PATH #전역 변수를 사용해서 경로를 기억
    
    model_id = url.replace("https://civitai.com/models/", "").split("/")[0]
    
    try:
        response = requests.get(f"https://civitai.com/api/v1/models/{model_id}")
    except Exception as e:
        print(f"[Error]{e}")
        raise e

    download_url = response.json()['modelVersions'][0]['downloadUrl']
    filename = response.json()['modelVersions'][0]['files'][0]['name']
    
    file_path = f"model/{filename}"
    if os.path.exists(file_path):
        print(f"[Error] File already exists: {file_path}")
        MODEL_PATH = file_path #모델 경로 기억
        return file_path
    
    os.makedirs("models", exist_ok=True)
    download_from_url(download_url, file_path)
    print(f"[Error] File downloades: {file_path}")
    
    #모델 경로 기억
    MODEL_PATH = file_path
    return file_path

# ./models 폴더에서 가장 최근에 수정된 모델 파일 찾기
def find_latest_model_in_directory(directory):
    model_files = glob.glob(f"{directory}/*.safetensors")
    if not model_files:
        return None
    
    # 가장 최근에 수정된 모델 파일 선택
    latest_model = max(model_files, key=os.path.getmime)
    return latest_model

In [17]:
app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7864

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on public URL: https://7fad217d741ca70461.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
